<a href="https://colab.research.google.com/github/Osbird/MachineLearningForKnowledgeService_KnowledgeEngineering/blob/master/%5Bhw5%5DRNN%20for%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KSE801, Machine Learning for knowledge service, HW5

- Magnus Myrmo Osberg



In [0]:
'''Setup of environment'''
!pip install -U nltk

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import nltk
nltk.download('all')

import pandas as pd
import numpy as np

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.4.1)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [0]:
torch.cuda.is_available()

True

In [0]:
from google.colab import drive
mountPoint = '/content/gdrive' 
drive.mount(mountPoint)

# Mounting of google drive in order to fetch the datasets

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
dataset = {}
# Please alter the path to adapt for where on google drive you have stored the data files
fullDrivePath = mountPoint+'/'

filenames = ['train.tsv','test.tsv']

for name in filenames:
  # Alter path accordingly
  path = fullDrivePath+name # using gDrive
  #path = filenames[name] # local files in same folder as .ipynb file
  
  print(path)
  key = name[0:-4] #stripping file extension from name
  dataset[key] = pd.read_csv(path, sep='\t', encoding='ascii', header=0)
  
for key in dataset:
  print (key+"\'s columns are: \n" + str(dataset[key].columns.tolist())+'\n')

# The columns of which to do the nlp preprocessing on
nlpColumns = {'train':['Phrase'],'test':['Phrase']} #Columns to perform cleaning



/content/gdrive/My Drive/Colab Notebooks/machine learning for knowledge service dataset/train.tsv
/content/gdrive/My Drive/Colab Notebooks/machine learning for knowledge service dataset/test.tsv
train's columns are: 
['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']

test's columns are: 
['PhraseId', 'SentenceId', 'Phrase']



---
### Load in and visualize the data

In [0]:
import numpy as np

reviews = dataset['train']['Phrase'].as_matrix()
labels = dataset['train']['Sentiment'].as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [0]:
test_reviews = dataset['test']['Phrase'].as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
len(labels)

156060

## Data pre-processing

In [0]:
from string import punctuation

# capitalize all words to lowercase
reviews = [word.lower() for word in reviews]

# rm punctuation
print(punctuation)
all_text = ' '.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [0]:
test_reviews = [word.lower() for word in test_reviews]

In [0]:
# verification
print(len(reviews))
print(reviews[0])
print(reviews[0].split())

156060
a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .
['a', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.']


In [0]:
# for generality, split by new lines and spaces
# this is not necessary for our data set
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [0]:
words[:10]

['a',
 'series',
 'of',
 'escapades',
 'demonstrating',
 'the',
 'adage',
 'that',
 'what',
 'is']

In [0]:
len(words)

1124143

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [0]:
from collections import Counter

# Dictionary maps word to int
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)} 

In [0]:
# tokenize each review in reviews_split
# store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews:
  reviews_ints.append([vocab_to_int[word] if word in vocab_to_int else 0 for word in review.split()])
  
test_reviews_ints = []
for review in test_reviews:
  test_reviews_ints.append([vocab_to_int[word] if word in vocab_to_int else 0 for word in review.split()])

In [0]:
# print tokens in first review
print('Tokenized review: \n', test_reviews_ints[:1])

Tokenized review: 
 [[18, 2481, 2898, 21, 572, 977, 379, 7]]


In [0]:
# verification
len(vocab_to_int.keys())

16531

**Verification of data**

In order to verify correct implementation of the dictionary, printing the number of unique words in the vocabulary and the contents of the first, tokenized review.

In [0]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))

Unique words:  16531



In [0]:
# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Tokenized review: 
 [[3, 315, 4, 15213, 5909, 1, 6508, 11, 56, 10, 53, 15, 1, 3516, 10, 177, 53, 15, 1, 11452, 2, 70, 4, 83, 627, 10513, 21, 589, 4, 83, 2013, 6, 60, 4, 3, 44, 7]]


In [0]:
encoded_labels = labels

---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [0]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
        
    ## for each review, I grab that review
    for i, row in enumerate(reviews_ints):
#       print(row)
      if len(row) > 0:
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [0]:
# Test your implementation!

seq_length = 32

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:5,:32])

[[    3   315     4 15213  5909     1  6508    11    56    10    53    15
      1  3516    10   177    53    15     1 11452     2    70     4    83
    627 10513    21   589     4    83  2013     6]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     3   315     4 15213  5909     1
   6508    11    56    10    53    15     1  3516]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     3   315]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     3]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   315]]


In [0]:
# Test your implementation!

seq_length = 32

test_features = pad_features(test_reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(test_features)==len(test_reviews_ints), "Your features should have as many rows as reviews."
assert len(test_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(test_features[:5,:32])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   18 2481 2898   21
   572  977  379    7]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   18 2481 2898
    21  572  977  379]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0   18]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0 2481 2898
    21  572  977  379]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0 2481
  2898   21  572  977]]


In [0]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeatures Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))



			Features Shapes:
Train set: 		(124848, 32) 
Validation set: 	(15606, 32) 
Test set: 		(15606, 32)


In [0]:
train_y[0]

1

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 100

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
# put labels in there
# This is really bad naming, but this is how it is.
real_test_data = TensorDataset(torch.from_numpy(test_features), torch.from_numpy(dataset['test']['PhraseId'].as_matrix()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
real_test_loader = DataLoader(real_test_data, shuffle=False, batch_size=batch_size)

In [0]:
type(dataset['test']['PhraseId'].values)

numpy.ndarray

In [0]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([100, 32])
Sample input: 
 tensor([[    0,     0,     0,  ...,     3, 12659,   560],
        [    0,     0,     0,  ...,     0,     0,  4700],
        [    0,     0,     0,  ...,     0,     0,   105],
        ...,
        [    0,     0,     0,  ...,  1077,   749,     7],
        [    0,     0,     0,  ...,     4,     1,   685],
        [    0,     0,     0,  ...,  1358,     4,  7370]])

Sample label size:  torch.Size([100])
Sample label: 
 tensor([3, 2, 2, 2, 3, 2, 2, 2, 3, 2, 1, 3, 2, 2, 1, 2, 1, 4, 3, 2, 4, 2, 2, 3,
        1, 2, 2, 2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 1, 2,
        3, 2, 2, 0, 2, 4, 2, 2, 3, 1, 2, 2, 4, 3, 2, 2, 2, 0, 3, 2, 3, 1, 2, 2,
        3, 2, 1, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1,
        2, 1, 2, 4])


---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="https://github.com/agungsantoso/deep-learning-v2-pytorch/blob/master/sentiment-rnn/assets/network_diagram.png?raw=1" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [0]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,  dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):

        batch_size = x.size(0)
        
        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden  = self.lstm(embeds)
        lstm_out = lstm_out.contiguous()
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # Slicing to get the last value out oft he LSTM.
        out = out[:, -1] # get last batch of labels 
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if(train_on_gpu):
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [0]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int) + 1 # +1 for zero padding + our word tokens
output_size = 5
embedding_dim = 300
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

epochs = 3
counter = 0
print_every = 500

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
for e in range(epochs):
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        net.zero_grad()
        output, _ = net(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, _ = net(inputs)
                val_loss = criterion(output, labels)
                val_losses.append(val_loss.item())

            net.train()

            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

SentimentRNN(
  (embedding): Embedding(16532, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=256, out_features=5, bias=True)
)
Epoch: 1/3... Step: 500... Loss: 1.222836... Val Loss: 1.110313
Epoch: 1/3... Step: 1000... Loss: 0.802983... Val Loss: 1.061298
Epoch: 2/3... Step: 1500... Loss: 0.765005... Val Loss: 1.044775
Epoch: 2/3... Step: 2000... Loss: 0.820981... Val Loss: 1.047434
Epoch: 3/3... Step: 2500... Loss: 0.704417... Val Loss: 1.017016
Epoch: 3/3... Step: 3000... Loss: 0.636835... Val Loss: 1.066112
Epoch: 3/3... Step: 3500... Loss: 0.677202... Val Loss: 1.053050


In [0]:
import os
out_path = './'

def test(model, real_test_loader, seed, mode):
    model.eval()
    pids, sentiments = [], []
    
    # predict sentiments for each phrase in test set
    for x, pid in real_test_loader:
        x = x.cuda()

        pred, _ = model(x)
        sentiment = pred.max(1)[1]

        pids += pid.tolist()
        sentiments += sentiment.tolist()
    
    # make a file for submission
    submission = pd.DataFrame({'PhraseId': pids, 'Sentiment': sentiments})
    submission_path = os.path.join(out_path, f'submission_{mode}_{seed}.csv')
    print(submission.head())
    submission.to_csv(submission_path, index=False)
    print(f'Saved predictions with the {mode} model to {submission_path}')
    
    return

In [0]:
test(net, real_test_loader, 0, "final")

   PhraseId  Sentiment
0    156061          2
1    156062          2
2    156063          2
3    156064          2
4    156065          2
Saved predictions with the final model to ./submission_final_0.csv


In [0]:
# Download the file.
from google.colab import files
files.download('./submission_final_0.csv')